In [1]:
import numpy as np
import pandas as pd
import os

# 불필요한 경고 표시 생략
import warnings
warnings.filterwarnings(action = 'ignore')

In [2]:
# 임의 데이터 생성
df = pd.DataFrame({
    '지역': ['서울', '서울', '서울', '경기', '경기', '부산', '서울', '서울', '부산', '경기', '경기', '경기'],
    '요일': ['월요일', '화요일', '수요일', '월요일', '화요일', '월요일', '목요일', '금요일', '화요일', '수요일', '목요일', '금요일'],
    '강수량': [100, 80, 1000, 200, 200, 100, 50, 100, 200, 100, 50, 100],
    '강수확률': [80, 70, 90, 10, 20, 30, 50, 90, 20, 80, 50, 10]
                  })
df

,지역,요일,강수량,강수확률
0,서울,월요일,100,80
1,서울,화요일,80,70
2,서울,수요일,1000,90
3,경기,월요일,200,10
4,경기,화요일,200,20
5,부산,월요일,100,30
6,서울,목요일,50,50
7,서울,금요일,100,90
8,부산,화요일,200,20
9,경기,수요일,100,80


- 위와 같은 형태의 DataFrame은 자료를 '지역별' 로 혹은 '요일별' 로 보고자 할때는 적합하지 않다.
- 이를 pivot 을 활용해 보기 좋게 가공할 수 있다.

# pivot 
 - dataframe의 형태를 변경하는 함수
 - 인덱스, 컬럼, 데이터로 사용할 컬럼을 명시한다.
```python
df.pivot(index, columns, values)
```

## index : 지역, column : 요일

In [3]:
df.pivot('지역','요일')

강수량                              강수확률                        
요일    금요일   목요일     수요일    월요일    화요일   금요일   목요일   수요일   월요일   화요일
지역                                                                 
경기  100.0  50.0   100.0  200.0  200.0  10.0  50.0  80.0  10.0  20.0
부산    NaN   NaN     NaN  100.0  200.0   NaN   NaN   NaN  30.0  20.0
서울  100.0  50.0  1000.0  100.0   80.0  90.0  50.0  90.0  80.0  70.0

## index : 요일, column : 지역

In [4]:
df.pivot('요일', '지역')

강수량                 강수확률            
지역      경기     부산      서울    경기    부산    서울
요일                                         
금요일  100.0    NaN   100.0  10.0   NaN  90.0
목요일   50.0    NaN    50.0  50.0   NaN  50.0
수요일  100.0    NaN  1000.0  80.0   NaN  90.0
월요일  200.0  100.0   100.0  10.0  30.0  80.0
화요일  200.0  200.0    80.0  20.0  20.0  70.0

## index : 요일, column : 지역 + '강수량' 만 보기

In [5]:
df.pivot('요일', '지역', '강수량')

지역,경기,부산,서울
요일,,,
금요일,100.0,NaN,100.0
목요일,50.0,NaN,50.0
수요일,100.0,NaN,1000.0
월요일,200.0,100.0,100.0
화요일,200.0,200.0,80.0


# pivot_table
 - 기능적으로 pivot과 동일
 - pivot과의 차이점
   - 중복되는 모호한 값이 있을 경우, aggregation 함수 사용하여 값을 채움(중복 데이터 처리가 가능)

In [6]:
# 기존 데이터에서 '서울 - 화요일'을  '서울 - 월요일' 로 바꿔 중복데이터 생성

df2 = pd.DataFrame({
    '지역': ['서울', '서울', '서울', '경기', '경기', '부산', '서울', '서울', '부산', '경기', '경기', '경기'],
    '요일': ['월요일', '월요일', '수요일', '월요일', '화요일', '월요일', '목요일', '금요일', '화요일', '수요일', '목요일', '금요일'],
    '강수량': [100, 80, 1000, 200, 200, 100, 50, 100, 200, 100, 50, 100],
    '강수확률': [80, 70, 90, 10, 20, 30, 50, 90, 20, 80, 50, 10]
                  })
df2

,지역,요일,강수량,강수확률
0,서울,월요일,100,80
1,서울,월요일,80,70
2,서울,수요일,1000,90
3,경기,월요일,200,10
4,경기,화요일,200,20
5,부산,월요일,100,30
6,서울,목요일,50,50
7,서울,금요일,100,90
8,부산,화요일,200,20
9,경기,수요일,100,80


In [7]:
# 중복데이터가 있을때 pivot은 에러가 발생한다.

# Index contains duplicate entries, cannot reshape 
df2.pivot('요일','지역')

ValueError: Index contains duplicate entries, cannot reshape

In [8]:
pd.pivot_table(df2, index='요일', columns='지역', aggfunc=np.mean)

강수량                 강수확률            
지역      경기     부산      서울    경기    부산    서울
요일                                         
금요일  100.0    NaN   100.0  10.0   NaN  90.0
목요일   50.0    NaN    50.0  50.0   NaN  50.0
수요일  100.0    NaN  1000.0  80.0   NaN  90.0
월요일  200.0  100.0    90.0  10.0  30.0  75.0
화요일  200.0  200.0     NaN  20.0  20.0   NaN